In [ ]:
from finlab import data
import finlab
from airflow.models import Variable
import os

finlab.login(Variable.get('FINLAB_API_KEY'))
db_path = os.path.join('/home/jovyan/work', 'finlab_data')
data.set_storage(data.FileStorage(path=db_path))


In [ ]:
from airflow.providers.apache.hive.hooks.hive import HiveServer2Hook
hive_hook = HiveServer2Hook(hiveserver2_conn_id='hive_cli_default')

In [ ]:
import pandas as pd

# 讀取CSV文件的頭部來獲取欄位名稱
data_sample = data.get('etl:adj_high')

def infer_data_type(value):
    if pd.isnull(value):
        return 'DOUBLE'  # 對於NaN值，假設欄位類型為DOUBLE
    try:
        float(value)
        return 'DOUBLE'
    except ValueError:
        return 'STRING'

# 生成CREATE TABLE語句
create_table_stmt = "CREATE EXTERNAL TABLE IF NOT EXISTS default.etl_adj_high (\n"
for column in data_sample.columns:
    # 對於以數字開頭的列名，加上反引號
    safe_column = f"`{column}`" if column[0].isdigit() else column
    data_type = infer_data_type(data_sample[column][0])
    create_table_stmt += f"    {safe_column} {data_type},\n"
create_table_stmt = create_table_stmt.rstrip(',\n') + "\n) STORED AS PARQUET LOCATION 'hdfs://namenode:9000/user/hue/finlab/data/etl_adj_high/'"

hive_hook.run(sql=create_table_stmt)
